# RFM Example

In [1]:
#IMPORT MODULES
import pandas as pd
import numpy as np
import datetime as dt

# visulisation 
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt



In [15]:
# load data
df_uk = pd.read_excel('UK_Data.xlsx',index_col=0)


In [7]:
# To calculate Recency value, we need to get a reference date
ref = df_uk.InvoiceDate.dt.date.max()
ref

datetime.date(2011, 12, 9)

In [16]:
# Since InvoiceDate column contains time values, we need to convert the column to date only values
df_uk.InvoiceDate = df_uk.InvoiceDate.dt.date
df_uk.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom,20.34


In [20]:
# We can then get Recency values for each customer by calculating the difference between the reference date (i.e., 2011-12-09) 
# and the most recent date when a customer made a purchase. Consider below, where we groupby CustomerID and apply a lambda function on InvoiceDate:
recency = df_uk.groupby('CustomerID')['InvoiceDate'].apply(lambda x: (ref-x.max()).days)
recency.head()

CustomerID
12747      2
12748      0
12749      3
12820      3
12821    214
Name: InvoiceDate, dtype: int64

In [ ]:
# To calculate Frequency values, we can use the total number of unique invoice number per customer. 
frequency = df_uk.groupby('CustomerID')['InvoiceNo'].nunique()
frequency.head()

In [ ]:
# For Monetary values, we can use the total amount spent by each customer over the entire period:
monetary = df_uk.groupby('CustomerID')['Amount'].sum()
monetary.head()

In [ ]:
# After getting all three R, F, and M values, we can combine them into one DataFrame.
# First combine recency and frequency
df_rf = recency.to_frame(name='Recency').\
join(frequency.to_frame(name='Frequency'))

# then monetary
df_rfm = df_rf.join(monetary.to_frame(name='Monetary'))
df_rfm.head()

In [ ]:
# The next step is to convert the respective R, F, and M values into scale scores. 
# We use quantiles to determine five equal sized groups, thereby assigning R, F, and M values on a 1–5 scale.
Quantiles = df_rfm.quantile(q=[0.2,0.4,0.6,0.8])
Quantiles

In [ ]:
# Based on these values, we can convert the three R, F, M values into scale scores.
def Rscore(x,q,df): #x=value, q=column, df = dataframe
    if x <= df[q][0.2]:
        return 5
    elif x <= df[q][0.4]:
        return 4
    elif x <= df[q][0.6]:
        return 3
    elif x<= df[q][0.8]:
        return 2
    else:
        return 1

In [ ]:
# Since we will use df_rfm for other purpose later, we shall create a copy of the DataFrame and then apply the function to derive R score. 
rfm_copy = df_rfm.copy()
rfm_copy['Rscore'] = rfm_copy['Recency'].\
apply(Rscore,args=('Recency',Quantiles))
rfm_copy.head()

In [ ]:
# Likewise, we can define a function for F and M scores conversion and call it to create Fscore and Mscore columns in the rfm_copy DataFrame
def FMscore (x,q,df):
    if x <= df[q][0.2]:
        return 1
    elif x <= df[q][0.4]:
        return 2
    elif x <= df[q][0.6]:
        return 3
    elif x<= df[q][0.8]:
        return 4
    else:
        return 5
    
rfm_copy['Fscore'] = rfm_copy['Frequency'].\
apply(FMscore,args=('Frequency',Quantiles))
rfm_copy['Mscore'] = rfm_copy['Monetary'].\
apply(FMscore,args=('Monetary',Quantiles))

rfm_copy.head(10)

In [ ]:
# For customer segmentation, we can use the aggregate RFM scores by simply taking the average of the R, F, M scores 
rfm_copy['RFMscore'] = rfm_copy[['Rscore','Fscore','Mscore']].mean(axis=1)
rfm_copy.head()

In [ ]:
# Based on the RFM score we can segment customers into different groups
def segment(df):
    if df.RFMscore <= 1:
        return 'Basic'
    elif df.RFMscore <= 2:
        return 'Bronze'
    elif df.RFMscore <= 3:
        return 'Silver'
    elif df.RFMscore <= 4:
        return 'Gold'
    elif df.RFMscore <= 4.5:
        return 'Platinum'
    else:
        return 'Diamond'
    


In [ ]:
# Apply function to get segmentation
rfm_copy['LoyaltyGroup'] = rfm_copy.apply(segment,axis=1)
rfm_copy.head(10)

In [ ]:
# check how many customers are in each group
rfm_score = rfm_copy.reset_index()
rfm_score.groupby('LoyaltyGroup')['CustomerID'].count().sort_values(ascending=False)